In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [4]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife-torch/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 32 commands: 
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_clip_imgsize_0_0"              --img_size=224 --substrate="nca" --init="zeros" --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros"    --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path=None --prompt="a red apple"                          --n_augs=4 --augs="crop+pers" --bs=4 --lr=0.001 --n_iters=100000
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_clip_imgsize_1_0"              --img_size=224 --substrate="nca" --init="zeros" --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros"    --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path=None --prompt="a very fat cat"                       --n_augs=4 --augs="crop+pers" --bs=4 --lr=0.001 --n_iters=100000
python clip_nca_optim_torch.py --seed=0 --save_dir="/hom

In [5]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43


In [6]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [7]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife-torch/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/job_00003.status
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_optim_torch/nca_clip_imgsize_3_0"              --img_size=224 --substrate="nca" --init="zeros" --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros"    --dt=0.01 --p_drop=0.0 --n_steps=64 --target_img_path=None --prompt="an artificial life simulation"        --n_augs=4 --augs="crop+pers" --bs=4 --lr=0.001 --n_iters=100000 &> /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/job_00003.status

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/job_00007.status
python clip_nca_optim_torch.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/clip_nca_

In [8]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_18_16_54_43/gpu_7.sh


# Track Experiment

In [93]:
a = []
for job_id in range(len(commands)):
    try:
        with open(f'{log_dir}/job_{job_id:05d}.status', 'r') as f:
            a.append(int(f.read().strip()))
    except Exception as e:
        print(job_id, e)
        a.append(-1)
a = np.array(a)

0 invalid literal for int() with base 10: ''
1 invalid literal for int() with base 10: ''
2 invalid literal for int() with base 10: ''
3 invalid literal for int() with base 10: ''
4 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00004.status'
5 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00005.status'
6 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00006.status'
7 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00007.status'
8 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00008.status'
9 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_51/job_00009.status'
10 [Errno 2] No such file or directory: '/home/akarshkumar0101/nca-alife-data/logs/2024_06_17_15_15_

In [94]:
a

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [95]:
np.array(a).mean()

-1.0

In [96]:
len(commands)

32

In [23]:
for job_id in range(len(commands)):
    print(job_id)
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        print(e)

0
100%|██████████| 100000/100000 [4:14:42<00:00,  6.54it/s, loss=-0.296]
1
100%|██████████| 100000/100000 [4:15:06<00:00,  6.53it/s, loss=-0.345]
2
100%|██████████| 100000/100000 [4:15:24<00:00,  6.53it/s, loss=-0.255]
3
100%|██████████| 100000/100000 [4:14:19<00:00,  6.55it/s, loss=-0.314]
4
100%|██████████| 100000/100000 [1:38:03<00:00, 17.00it/s, loss=-0.364]
5
100%|██████████| 100000/100000 [1:41:57<00:00, 16.35it/s, loss=-0.26]
6
100%|██████████| 100000/100000 [1:45:26<00:00, 15.81it/s, loss=-0.352]
7
100%|██████████| 100000/100000 [1:38:18<00:00, 16.95it/s, loss=-0.315]
8
100%|██████████| 100000/100000 [3:23:34<00:00,  8.19it/s, loss=-0.423]
9
100%|██████████| 100000/100000 [3:08:52<00:00,  8.82it/s, loss=-0.416]
10
100%|██████████| 100000/100000 [3:20:47<00:00,  8.30it/s, loss=-0.258]
11
100%|██████████| 100000/100000 [2:51:11<00:00,  9.74it/s, loss=-0.394]
12
100%|██████████| 100000/100000 [2:59:32<00:00,  9.28it/s, loss=-0.372]
13
100%|██████████| 100000/100000 [2:57:36<00:00,